# Implementing Proximal Policy Optimization 


In this notebook you will be implementing Proximal Policy Optimization algorithm, 
scaled up version of which was used to train [OpenAI Five](https://openai.com/blog/openai-five/) 
to [win](https://openai.com/blog/how-to-train-your-openai-five/) against the
world champions in Dota 2.
You will be solving a continuous control environment on which it may be easier and faster 
to train an agent, however note that PPO here may not be the best algorithm as, for example,
Deep Deterministic Policy Gradient and Soft Actor Critic may be more suited 
for continuous control environments. To run the environment you will need to install 
[pybullet-gym](https://github.com/benelot/pybullet-gym) which unlike MuJoCo 
does not require you to have a license.

**If you have some weird troubles with pybullet, try ver.2.5.6 : `pip install pybullet==2.5.6`**

To install the library:

In [ ]:
!git clone https://github.com/benelot/pybullet-gym lib/pybullet-gym
!pip install -e lib/pybullet-gym

In [ ]:
import gym 
import pybulletgym

The overall structure of the code is similar to the one in the A2C optional homework, but don't worry if you haven't done it, it should be relatively easy to figure it out. 

First, we will create an instance of the environment. The first wrapper will simply write summaries, mainly, the total reward during an episode. Then we will *normalize* the observations and rewards: subtract running mean from observations and rewards and divide the resulting quantities by the running variances.

In [ ]:
from mujoco_wrappers import Normalize
from logger import TensorboardSummaries as Summaries

env = gym.make("HalfCheetahMuJoCoEnv-v0")
env = Normalize(Summaries(env));
env.unwrapped.seed(0);

print("observation space: ", env.observation_space,
      "\nobservations:", env.reset())
print("action space: ", env.action_space, 
      "\naction_sample: ", env.action_space.sample())

Next, you will need to define a model for training. You can use two separate networks (one for policy and another for value function) or create one with shared backbone. Recommended architecture is a 3-layer MLP with 64 hidden units, $\mathrm{tanh}$ 
activation function, weights initialized with orthogonal initializer with gain $\sqrt{2}$ and biases initialized with zeros. 

Our policy distribution is going to be multivariate normal with diagonal covariance. The policy head will predict the mean and covariance OR only mean, and the covariance then should be represented by a single (learned) vector of size 6 (corresponding to the dimensionality of the action space from above), initialized with zeros. Anyway you should guarantee that covariance is non-negative by using exponent or softplus. 

Overall the model should return three things: predicted mean of the distribution, variance vector and value estimation. 

In [ ]:
import torch

<Define your model here>

'''
input:
    states - tensor, (batch_size x features)
output:
    mean - tensor, (batch_size x actions_dim)
    cov - tensor, (batch_size x actions_dim)
    V - tensor, critic estimation, (batch_size)
'''

This model will be wrapped by a `Policy`. The policy can work in two modes, but in either case 
it is going to return dictionary with string-type keys. The first mode is when the policy is 
used to sample actions for a trajectory which will later be used for training. In this case 
the flag `training` passed to `act` method is `False` and the method should return 
a `dict` with the following keys: 

* `"actions"`: actions to pass to the environment
* `"log_probs"`: log-probabilities of sampled actions
* `"values"`: value function $V^\pi(s)$ predictions.

We don't need to use the values under these keys for training, so all of them should be of type `np.ndarray`. This regime will be used to collect data.

When `training` is `True`, the model is training on a given batch of observations. In this
case it should return a `dict` with the following keys

* `"distribution"`: an instance of multivariate normal distribution (`torch.distributions.MultivariateNormal`)
* `"values"`: value function $V^\pi(s)$ prediction, tensor

The distinction about the modes comes into play depending on where the policy is used: if it is called from `EnvRunner`, 
the `training` flag is `False`, if it is called from `PPO`, the `training` flag is `True`. These classes 
will be described below.

In [ ]:
from torch.distributions import MultivariateNormal
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

class Policy:
    def __init__(self, model):
        self.model = model

    def act(self, inputs, training=False):
        '''
        input:
            inputs - numpy array if training is False, otherwise tensor, (batch_size x features)
            training - flag, bool
        output:
            if training is True, dict containing keys ['actions', 'log_probs', 'values']:
                `distribution` - MultivariateNormal, (batch_size x actions_dim)
                'values' - critic estimations, tensor, (batch_size)
            if training is False, dict containing keys ['actions', 'log_probs', 'values']:
                'actions' - selected actions, numpy, (batch_size)
                'log_probs' - log probs of selected actions, numpy, (batch_size)
                'values' - critic estimations, numpy, (batch_size)
        '''
        # if training is false, input is numpy
        if not training:
            inputs = torch.FloatTensor(inputs).to(DEVICE)            
        
        <YOUR CODE>
        
        if training:
            return {
                "distribution": <>,
                "values": <>
            }
        else:
            return {
                "actions": <>,
                "log_probs": <>,
                "values": <>
            }

We will use `EnvRunner` to perform interactions with an environment with a policy for a fixed number of timesteps. Calling `.get_next()` on a runner will return a trajectory &mdash; dictionary 
containing keys

* `"observations"`
* `"rewards"` 
* `"dones"`
* `"actions"`
* all other keys that you defined in `Policy` in `training=False` regime,

under each of these keys there is a `np.ndarray` of specified length $T$ &mdash; the size of partial trajectory. 

In [ ]:
from runners import EnvRunner

Additionally, before returning a trajectory this runner can apply a list of transformations. 
Each transformation is simply a callable that should modify passed trajectory in-place.

In [ ]:
import numpy as np

class AsArray:
    """ 
    Converts lists of interactions to ndarray.
    """
    def __call__(self, trajectory, last_observation):
        # Modifies trajectory inplace.
        # Just switches python lists to numpy arrays
        for k, v in trajectory.items():
            trajectory[k] = np.asarray(v)

Let's have a look at how this works.

In [ ]:
model = <init your model>
policy = Policy(model)
runner = EnvRunner(env, policy, nsteps=5, transforms=[AsArray()])

In [ ]:
# generates new rollout
trajectory = runner.get_next()

In [ ]:
# what is inside
print(trajectory.keys())

In [ ]:
# Sanity checks
assert 'log_probs' in trajectory, "Not found: policy didn't provide log_probs of selected actions"
assert 'values' in trajectory, "Not found: policy didn't provide critic estimations"
assert trajectory['log_probs'].shape == (5,), "log_probs wrong shape"
assert trajectory['values'].shape == (5,), "values wrong shape"
assert trajectory['observations'].shape == (5, 17), "observations wrong shape"
assert trajectory['rewards'].shape == (5,), "rewards wrong shape"
assert trajectory['dones'].shape == (5,), "dones wrong shape"

In [ ]:
# Here is what collected inside
{k: v.shape for k, v in trajectory.items()}

You will need to implement the following two transformations. 

The first is `GAE` that implements [Generalized Advantage Estimator](https://arxiv.org/abs/1506.02438).
In it you should add two keys to the trajectory: `"advantages"` and `"value_targets"`. In GAE the advantages
$A_t^{\mathrm{GAE}(\gamma,\lambda)}$ are essentially defined as the exponential 
moving average with parameter $\lambda$ of the regular advantages 
$\hat{A}^{(T)}(s_t) = \sum_{l=0}^{T-1-t} \gamma^l r_{t+l} + \gamma^{T} V^\pi(s_{T}) - V^\pi(s_t)$. 
The exact formula for the computation is the following

$$
A_{t}^{\mathrm{GAE}(\gamma,\lambda)} = \sum_{l=0}^{T-1-t} (\gamma\lambda)^l\delta_{t + l}^V, \, t \in [0, T)
$$
where $\delta_{t+l}^V = r_{t+l} + \gamma V^\pi(s_{t+l+1}) - V^\pi(s_{t+l})$. You can look at the 
derivation (formulas 11-16) in the paper. Don't forget to reset the summation on terminal
states as determined by the flags `trajectory["dones"]`. You can use `trajectory["values"]`
to get values of all observations except the most recent which is stored under 
 `trajectory["state"]["latest_observation"]`. For this observation you will need to call the policy 
 to get the value prediction.

Once you computed the advantages, you can get the targets for training the value function by adding 
back values:
$$
\hat{V}(s_{t+l}) = A_{t+l}^{\mathrm{GAE}(\gamma,\lambda)} + V(s_{t + l}),
$$
where $\hat{V}$ is a tensor of value targets that are used to train the value function. 

In [ ]:
class GAE:
    """ Generalized Advantage Estimator. """
    def __init__(self, policy, gamma=0.99, lambda_=0.95):
        self.policy = policy
        self.gamma = gamma
        self.lambda_ = lambda_

    def __call__(self, trajectory, last_observation):
        '''
        This method should modify trajectory inplace by adding 
        items with keys 'advantages' and 'value_targets' to it
        
        input:
            trajectory - dict from runner
            latest_observation - last state, numpy, (features)
        '''
        <YOUR CODE>
            
        trajectory["advantages"] = <>
        trajectory["value_targets"] = <>

Let's run a small test just in case.

In [ ]:
# tests
class DummyEnv():
    def __init__(self):
        self.unwrapped = None
        self.t = 0
        self.state = np.zeros(17)
    
    def reset(self):
        return self.state
    
    def step(self, a):
        r = [0, -100, 800][self.t]
        done = self.t == 2
        self.t = (self.t + 1) % 3
        return self.state, r, done, {}
    
class DummyPolicy():
    def act(self, s):
        return {"values": np.array(100), "actions": np.array([-0.42, 0.42])}

dummy_env = DummyEnv()
dummy_policy = DummyPolicy()
runner = EnvRunner(dummy_env, dummy_policy, nsteps=8, transforms=[AsArray(), GAE(dummy_policy, gamma=0.8, lambda_=0.5)])
trajectory = runner.get_next()

In [ ]:
assert 'advantages' in trajectory, "Not found: advantage estimation"
assert 'value_targets' in trajectory, "Not found: targets for critic"
assert trajectory['advantages'].shape == (8,), "advantage wrong shape"
assert trajectory['value_targets'].shape == (8,), "value_targets wrong shape"
assert (trajectory['advantages'] == np.array([44, 160, 700, 44, 160, 700, -68, -120])).all(), "advantage computation error"
assert (trajectory['value_targets'] == trajectory['advantages'] + 100).all(), "value targets computation error"
print("Nice!")

The main advantage of PPO over simpler policy based methods like A2C is that it is possible
to train on the same trajectory for multiple gradient steps. The following class wraps 
an `EnvRunner`. It should call the runner to get a trajectory, then return minibatches 
from it for a number of epochs, shuffling the data before each epoch. The number of minibatches per epoch is predefined in `num_minibatches`.

In [ ]:
class Sampler:
    """ Samples minibatches from trajectory for a number of epochs. """
    def __init__(self, runner, num_epochs, num_minibatches, transforms=None):
        self.runner = runner
        self.num_epochs = num_epochs
        self.num_minibatches = num_minibatches
        self.transforms = transforms or []

    def get_next(self):
        """ 
        Yields next minibatch (dict) for training with at least following keys:
                'observations' - numpy, (batch_size x features)
                'actions' - numpy, (batch_size x actions_dim)
                'advantages' - numpy, (batch_size)
                'log_probs' - numpy, (batch_size)
        """
        trajectory = self.runner.get_next()
        
        for epoch in range(self.num_epochs):
            # shuffle dataset and separate it into minibatches
            # you can use any standard utils to do that
            <YOUR CODE>
            
            for _ in range(self.num_minibatches):
                <YOUR CODE>
                
                # applying additional transforms
                for transform in self.transforms:
                    transform(minibatch)
                
                yield minibatch

A common trick to use with GAE is to normalize advantages for every minibatch, the following transformation does that. 

In [ ]:
class NormalizeAdvantages:
    """ Normalizes advantages to have zero mean and variance 1. """
    def __call__(self, batch):
        adv = batch["advantages"]
        adv = (adv - adv.mean()) / (adv.std() + 1e-8)
        batch["advantages"] = adv

In [ ]:
class PyTorchify:
    """ Moves everything to PyTorch """
    def __call__(self, batch):
        for k, v in batch.items():
            batch[k] = torch.FloatTensor(v).to(DEVICE)

Finally, we can create our PPO runner! This is our pipeline of data collecting and generating mini-batches for our trainer!

In [ ]:
def make_ppo_sampler(env, policy, num_runner_steps=2048, gamma=0.99, lambda_=0.95, num_epochs=10, num_minibatches=32):
    """ Creates runner for PPO algorithm. """
    runner_transforms = [AsArray(), GAE(policy, gamma=gamma, lambda_=lambda_)]
    runner = EnvRunner(env, policy, num_runner_steps, transforms=runner_transforms)

    sampler_transforms = [NormalizeAdvantages(), PyTorchify()]
    sampler = Sampler(runner, num_epochs=num_epochs, 
                              num_minibatches=num_minibatches,
                              transforms=sampler_transforms)
    return sampler

In the next cell you will need to implement Proximal Policy Optimization algorithm itself. The algorithm
modifies the typical policy gradient loss in the following way:

$$
J_{\pi}(s, a) = \frac{\pi_\theta(a|s)}{\pi_\theta^{\text{old}}(a|s)} \cdot A^{\mathrm{GAE}(\gamma,\lambda)}(s, a)
$$

$$
J_{\pi}^{\text{clipped}}(s, a) = \mathrm{clip}\left(
\frac{\pi_\theta(a|s)}{\pi_{\theta^{\text{old}}}(a|s)},
1 - \text{cliprange}, 1 + \text{cliprange}\right)\cdot A^{\mathrm{GAE(\gamma, \lambda)}}(s)\\
$$

$$
L_{\text{policy}} = -\frac{1}{T}\sum_{l=0}^{T-1}\min\left(J_\pi(s_{t + l}, a_{t + l}), J_{\pi}^{\text{clipped}}(s_{t + l}, a_{t + l})\right).
$$

The value loss is also modified:

$$
L_{V}^{\text{clipped}} = \frac{1}{T}\sum_{l=0}^{T-1} \max(l^{simple}(s_{t + l}), l^{clipped}(s_{t + l})),
$$
where $l^{simple}$ is your standard critic loss
$$
l^{simple}(s_{t + l}) = [V_\theta(s_{t+l}) - \hat{V}(s_{t + l})]^2
$$

and $l^{clipped}$ is a clipped version that limits large changes of the value function:
$$
l^{clipped}(s_{t + l}) = [
V_{\theta^{\text{old}}}(s_{t+l}) +
\text{clip}\left(
V_\theta(s_{t+l}) - V_{\theta^\text{old}}(s_{t+l}),
-\text{cliprange}, \text{cliprange}
\right) - \hat{V}(s_{t + l})] ^ 2
$$

In [ ]:
from torch.nn.utils import clip_grad_norm_

class PPO:
    def __init__(self, policy, optimizer, sampler, cliprange=0.2, value_loss_coef=0.25, max_grad_norm=0.5):
        self.policy = policy
        self.optimizer = optimizer
        self.sampler = sampler
        self.cliprange = cliprange
        self.value_loss_coef = value_loss_coef
        self.max_grad_norm = max_grad_norm
        self.iteration = 0
        
    def write(self, name, val):
        """ For logging purposes """
        self.sampler.runner.write(name, val, self.iteration)

    def policy_loss(self, batch, act):
        """
        Computes and returns policy loss on a given minibatch.
        input:
            batch - dict from sampler, containing:
                'advantages' - advantage estimation, tensor, (batch_size)
                'actions' - actions selected in real trajectory, (batch_size)
                'log_probs' - probabilities of actions from policy used to collect this trajectory, (batch_size)
            act - dict from your current policy, containing:
                'distribution' - MultivariateNormal, (batch_size x actions_dim)
        output:
            policy loss - torch scalar
        """    
        <YOUR CODE>
        
        # additional logs: entropy, fraction of samples that were clipped, max ratio
        self.write('additional/entropy', <>)
        self.write('additional/policy_loss_clipped_fraction', <>)
        self.write('additional/max_ratio', <>)

    def value_loss(self, batch, act):
        """
        Computes and returns policy loss on a given minibatch.
        input:
            batch - dict from sampler, containing:
                'value_targets' - computed targets for critic, (batch_size)
                'values' - critic estimation from network that generated trajectory, (batch_size)
            act - dict from your current policy, containing:
                'values' - current critic estimation, tensor, (batch_size)
        output:
            critic loss - torch scalar
        """    
        <YOUR CODE>
        
        # additional logs: average value predictions, fraction of samples that were clipped
        self.write('additional/value_predictions', <>)
        self.write('additional/value_loss_clipped_fraction', <>)

    def loss(self, batch):
        """Computes loss for current batch"""
        
        # let's run our current policy on this batch 
        act = self.policy.act(batch["observations"], training=True)
        
        # compute losses
        # note that we don't need entropy regularization for this env.
        <YOUR CODE>      
                
        # log all losses
        self.write('losses', {
            'policy loss': <>,
            'critic loss': <>
        })
        
        # Return scalar loss        
        return <>

    def step(self, batch):
        """ Computes the loss function and performs a single gradient step for this batch. """
        <YOUR CODE>
        
        # do not forget to clip gradients using self.max_grad_norm
        # and log gradient norm
        self.write('gradient norm', <>)
        
        # this is for logging
        self.iteration += 1

Now everything is ready to do training. In **one million of interactions** it should be possible to 
achieve the average raw reward over last 100 episodes more than 1500. **Your goal is to reach 1000**.

For optimization it is suggested to use Adam optimizer with learning rate 3e-4 and epsilon 1e-5.

**Notes**:
* reward should rise quickly in this environment. If it is stuck, something went wrong.
* you can linearly decay learning rate if you face instabilities.
* we wrote additional logs with respect to *number of network updates* since we perform many updates after each data collection stage.

In [ ]:
model = <init your model>
policy = Policy(model)
sampler = make_ppo_sampler(env, policy)

optimizer = <choose your fighter>
ppo = PPO(policy, optimizer, sampler)

In [ ]:
<YOUR CODE>

**Note:** remember that we learned not only the weights of policy model, but also statistics for input normalization, so to save our results we need to store them too.

In [ ]:
# save your model just in case

def save(model, env, name):
    torch.save(model.state_dict(), name)
    np.save(name + "_mean_ob", env.obs_rmv.mean)
    np.save(name + "_var_ob", env.obs_rmv.var)
    np.save(name + "_count_ob", env.obs_rmv.count)
    
save(model, env, "PPO")

In [ ]:
# loading the model

def load(model, env, name):
    model.load_state_dict(torch.load(name))
    env.obs_rmv.mean = np.load(name + "_mean_ob.npy")
    env.obs_rmv.var = np.load(name + "_var_ob.npy")
    env.obs_rmv.count = np.load(name + "_count_ob.npy")
    
load(model, env, "PPO")

## Evaluation

In [ ]:
def evaluate(env, policy, n_games=1, t_max=1000):
    '''
    Plays n_games and returns rewards and rendered games
    '''
    rewards = []
    
    for _ in range(n_games):
        s = env.reset()
        
        R = 0
        for _ in range(t_max):
            action = policy.act(np.array([s]))["actions"][0]
            
            s, _, done, info = env.step(action)
            
            # remember that we used a wrapper that normalizes reward
            # original reward per step comes here
            R += info["original reward"]
                
            if done:
                break

        rewards.append(R)
    return np.array(rewards)

In [ ]:
# evaluation will take some time!
sessions = evaluate(env, policy, n_games=20)
score = sessions.mean()
print(f"Your score: {score}")

assert score >= 1000, "Needs more training?"
print("Well done!")

## Record

In [ ]:
# let's hope this will work
# don't forget to pray
env = gym.wrappers.Monitor(env, directory="videos", force=True)

In [ ]:
# record sessions
# note that t_max is 300, so collected reward will be smaller than 1000
evaluate(env, policy, n_games=1, t_max=300)

In [ ]:
env.close()